[@LorenaABarba](https://twitter.com/LorenaABarba)

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

***12 étapes vers Navier–Stokes***
======
***

Cette leçon complète le premier module interactif du cours en ligne [CFD Python](https://github.com/barbagroup/CFDPython), par le professeur Lorena A. Barba, intitulé **12 étapes vers Navier–Stokes.** Il a été écrit avec Gilbert Forsyth, étudiant diplômé de Boston University (BU).

Opérations sur les tableaux avec NumPy
----------------

Pour les programmes plus intensifs en calcul, l'utilisation des fonctions Numpy intégrées peut fournir une augmentation de la vitesse d'exécution plusieurs fois. À titre d'exemple simple, considérons l'équation suivante :

$$u^{n+1}_i = u^n_i-u^n_{i-1}$$

Maintenant, étant donné un vecteur $u^n = [0, 1, 2, 3, 4, 5]\ \ $ nous pouvons calculer les valeurs de $u^{n+1}$ en itérant sur les valeurs de $u^ n$ avec une boucle for.

In [2]:
import numpy

In [3]:
u = numpy.array((0, 1, 2, 3, 4, 5))

for i in range(1, len(u)):
    print(u[i] - u[i-1])

1
1
1
1
1


C'est le résultat attendu et le temps d'exécution a été quasi instantané. Si nous effectuons la même opération qu'une opération de tableau, alors plutôt que de calculer $u^n_i-u^n_{i-1}\ $ 5 fois séparément, nous pouvons découper le tableau $u$ et calculer chaque opération avec une seule commande :

In [4]:
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

Ce que dit cette commande est de soustraire les 0e, 1er, 2e, 3e, 4e et 5e éléments de $u$ des 1er, 2e, 3e, 4e, 5e et 6e éléments de $u$.

### La vitesse augmente

Pour un tableau à 6 éléments, les avantages des opérations de tableau sont assez minces. Il n'y aura pas de différence appréciable dans le temps d'exécution car il y a si peu d'opérations en cours. Mais si nous revisitons la convection linéaire 2D, nous pouvons voir des augmentations de vitesse substantielles.

In [5]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .2
dt = sigma * dx

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

u = numpy.ones((ny, nx)) ##create a 1xn vector of 1's
un = numpy.ones((ny, nx)) 

###Assign initial conditions

u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

Une fois nos conditions initiales configurées, essayons d'abord d'exécuter notre code de boucle imbriquée d'origine, en utilisant la fonction "magique" iPython `%%timeit`, qui nous aidera à évaluer les performances de notre code.

**Remarque** : La fonction magique `%%timeit` exécutera le code plusieurs fois et donnera ensuite un temps d'exécution moyen en conséquence. Si vous avez des chiffres tracés dans une cellule où vous exécutez `%%timeit`, il tracera ces chiffres à plusieurs reprises, ce qui peut être un peu compliqué.

Les temps d'exécution ci-dessous varient d'une machine à l'autre. Ne vous attendez pas à ce que vos temps correspondent à ces temps, mais vous _devriez_ vous attendre à voir la même tendance générale à la diminution du temps d'exécution lorsque nous passons aux opérations sur les tableaux.

In [6]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt + 1): ##loop across number of time steps
    un = u.copy()
    row, col = u.shape
    for j in range(1, row):
        for i in range(1, col):
            u[j, i] = (un[j, i] - (c * dt / dx * 
                                  (un[j, i] - un[j, i - 1])) - 
                                  (c * dt / dy * 
                                   (un[j, i] - un[j - 1, i])))
            u[0, :] = 1
            u[-1, :] = 1
            u[:, 0] = 1
            u[:, -1] = 1

1.79 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Avec le code Python "brut" ci-dessus, le temps d'exécution moyen atteint était de 3,07 secondes (sur un MacBook Pro mi 2012). Gardez à l'esprit qu'avec ces trois boucles imbriquées, les instructions à l'intérieur de la boucle **j** sont évaluées plus de 650 000 fois. Comparons cela avec les performances du même code implémenté avec des opérations de tableau :

In [7]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt + 1): ##loop across number of time steps
    un = u.copy()
    u[1:, 1:] = (un[1:, 1:] - (c * dt / dx * (un[1:, 1:] - un[1:, 0:-1])) -
                              (c * dt / dy * (un[1:, 1:] - un[0:-1, 1:])))
    u[0, :] = 1
    u[-1, :] = 1
    u[:, 0] = 1
    u[:, -1] = 1

4.39 ms ± 264 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Comme vous pouvez le voir, l'augmentation de la vitesse est substantielle. Le même calcul passe de 3,07 secondes à 7,38 millisecondes. 3 secondes n'est pas un temps d'attente énorme, mais ces gains de vitesse augmenteront de façon exponentielle avec la taille et la complexité du problème évalué.

In [9]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()